# Exploration du site Guts of Darkness

🎯 Objectif : 
Explorer la structure HTML d'une page d'album pour en extraire :
- Le nom de l'artiste
- Le nom de l'album
- Le texte de la critique
- Les styles (tags)
- Les notes (si disponible)


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import random
import re
import os


HEADERS = {
    "User-Agent": "MusicRecommenderBot/0.1 (+mailto:ton.email@example.com)"
}


In [7]:
def parse_album(html: str, url: str) -> dict:
    soup = BeautifulSoup(html, "html.parser")

    # === TITRE & ARTISTE ===
    h1 = soup.select_one("h1")
    title = h1.find("em").get_text(strip=True) if h1 and h1.find("em") else None
    artist = h1.get_text(" ", strip=True).replace(title, "").replace(">", "").strip() if h1 else None

    # Alternative : line-up
    lineup = soup.select_one("#objetLineup p")
    if lineup and not artist:
        artist = lineup.get_text(" ", strip=True)

    # === INFORMATIONS ===
    info_div = soup.select_one("#objet-informations")
    infos = []
    if info_div:
        infos = [p.get_text(" ", strip=True) for p in info_div.find_all("p")]

    # === CHRONIQUE ===
    chronique_div = soup.select_one("div.objet-chronique")
    chronique_text = ""
    if chronique_div:
        ps = chronique_div.find_all("p")
        chronique_text = " ".join(p.get_text(" ", strip=True) for p in ps)
        chronique_text = re.sub(r"\s+", " ", chronique_text).strip()

    # === STYLES ===
    style_div = soup.select_one("div.objet-style")
    styles = [a.get_text(strip=True) for a in style_div.find_all("a")] if style_div else []

    # === NOTE DE LA CHRONIQUE ===
    sous_chronique_div = soup.select_one("div.objet-sous-chronique div.discrete-info")
    note_chronique = None
    if sous_chronique_div:
        pleines = len(sous_chronique_div.select("span.gfxNotePleine"))
        demi = len(sous_chronique_div.select("span.gfxNoteDemi"))
        vide = len(sous_chronique_div.select("span.gfxNoteVide"))
        note_chronique = pleines + 0.5 * demi

    # === ALBUMS "DANS LE MÊME ESPRIT" ===
    related_section = soup.select("div.mosaique a h1 em")
    same_spirit = [em.get_text(strip=True) for em in related_section] if related_section else []

    # === NOTE MOYENNE ===
    vote_div = soup.select_one("div#objetVote")
    note_moyenne = None
    if vote_div:
        pleines = len(vote_div.select("span.gfxNotePleine"))
        demi = len(vote_div.select("span.gfxNoteDemi"))
        note_moyenne = pleines + 0.5 * demi

    # === TAGS (ajoutés par utilisateurs) ===
    tags_div = soup.select_one("div#contenuObjetTags")
    tags_text = ""
    if tags_div:
        tags_text = tags_div.get_text(" ", strip=True)

    return {
        "album_name": title,
        "artist_name": artist,
        "lineup": lineup.get_text(" ", strip=True) if lineup else None,
        "informations": " ".join(infos),
        "chronique": chronique_text,
        "styles": ";".join(styles),
        "note_chronique": note_chronique,
        "note_moyenne": note_moyenne,
        "same_spirit": ";".join(same_spirit),
        "tags_text": tags_text,
        "source_url": url,
    }

### Générer une liste d'URLS d'albums à partir des IDs numériques 
Exemple : start_id = 2400, end_id = 24100 -> 100 liens 


In [8]:
def generate_album_links(start_id: int, end_id: int):
    BASE_URL = "https://www.gutsofdarkness.com/god/objet.php?objet="
    urls = [f"{BASE_URL}{i}" for i in range(start_id, end_id + 1)]
    print(f" {len(urls)} liens générés ({urls[0]} → {urls[-1]})")
    return urls

### Multithreading pour scraper plusieurs pages en parallèle

In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def scrape_url(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        if r.status_code != 200:
            return None

        parsed = parse_album(r.text, url)
        if parsed["album_name"] and parsed["artist_name"]:
            return parsed
        return None
    except Exception:
        return None


### Applications du parse sur tous les liens générer

In [ ]:

urls = generate_album_links(00000, 25000)

output_path = "../data/processed/sample_albums.csv"

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    scraped_urls = set(df_existing["source_url"])
    print(f" {len(scraped_urls)} albums déjà présents — ils seront ignorés.")
else:
    scraped_urls = set()
    print(" Aucun dataset existant, scraping complet.")


# === Version multithread conservant la logique de mise à jour ===
urls_to_scrape = [u for u in urls if u not in scraped_urls]
print(f" === Lancement du scraping multithread ({len(urls_to_scrape)} nouvelles URLs)... === ")

rows = []
max_threads = min(32, (os.cpu_count() or 1) * 2) # adapter selon mon CPU / réseau

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = {executor.submit(scrape_url, url): url for url in urls_to_scrape}
    for i, future in enumerate(as_completed(futures), 1):
        result = future.result()
        if result:
            rows.append(result)
            print(f"[{i}] {result['artist_name']} - {result['album_name']}")
        else:
            print(f"[{i}] Échec ou page vide")

        # Sauvegarde intermédiaire tous les 100 albums valides
        if i % 100 == 0 and rows:
            temp_df = pd.DataFrame(rows)
            if os.path.exists(output_path):
                df_existing = pd.read_csv(output_path)
                temp_df = pd.concat([df_existing, temp_df], ignore_index=True)
                temp_df.drop_duplicates(subset=["source_url"], inplace=True)
            temp_df.to_csv(output_path, index=False, encoding="utf-8")
            print(f" === Sauvegarde intermédiaire ({len(temp_df)} albums) → {output_path} ===")
            rows = []  # on vide la mémoire

df = pd.DataFrame(rows)
print("\n=== Aperçu des données ===")
display(df.head())

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df = pd.concat([df_existing, df], ignore_index=True)

# Nettoyage avant export
df.drop_duplicates(subset=["source_url"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n Données exportées vers {output_path} ({len(df)} albums uniques)")

new_albums_count = len(df) - len(df_existing) if os.path.exists(output_path) else len(df)
print(f"\n {new_albums_count} nouveaux albums ajoutés. Total : {len(df)} albums uniques.")



 25001 liens générés (https://www.gutsofdarkness.com/god/objet.php?objet=0 → https://www.gutsofdarkness.com/god/objet.php?objet=25000)
 10126 albums déjà présents — ils seront ignorés.
 === Lancement du scraping multithread (14875 nouvelles URLs)... === 
[1] Échec ou page vide
[2] Échec ou page vide
[3] Échec ou page vide
[4] Échec ou page vide
[5] Échec ou page vide
[6] Échec ou page vide
[7] Échec ou page vide
[8] Échec ou page vide
[9] Échec ou page vide
[10] Échec ou page vide
[11] Échec ou page vide
[12] Échec ou page vide
[13] Échec ou page vide
[14] Échec ou page vide
[15] Échec ou page vide
[16] Échec ou page vide
[17] Échec ou page vide
[18] Échec ou page vide
[19] Échec ou page vide
[20] Échec ou page vide
[21] Échec ou page vide
[22] Échec ou page vide
[23] Échec ou page vide
[24] Échec ou page vide
[25] Échec ou page vide
[26] Échec ou page vide
[27] Échec ou page vide
[28] Échec ou page vide
[29] Échec ou page vide
[30] Échec ou page vide
[31] Échec ou page vide
[32] Échec